# High-level CNN TF Example

*Modified by Jordan A Caraballo Vega (jordancaraballo)*

In [1]:
import numpy as np
import os
import sys

import warnings # make notebook more readable and nice
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

import tensorflow as tf
from common.params import *
from common.utils import *

In [2]:
# Force one-gpu
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
# Performance Improvement
# 1. Auto-tune
# This is no longer needed with recent versions of TF
# And actually seems to make the performance worse
#os.environ['TF_ENABLE_WINOGRAD_NONFUSED'] = "1"

In [4]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Numpy: ", np.__version__)
print("Tensorflow: ", tf.__version__)
print("GPU: ", get_gpu_name())
print(get_cuda_version())
print("CuDNN Version ", get_cudnn_version())

OS:  linux
Python:  3.6.7 | packaged by conda-forge | (default, Jul  2 2019, 02:18:42) 
[GCC 7.3.0]
Numpy:  1.17.0
Tensorflow:  1.13.1
GPU:  ['Tesla V100-SXM2-16GB', 'Tesla V100-SXM2-16GB', 'Tesla V100-SXM2-16GB', 'Tesla V100-SXM2-16GB']
CUDA Version 10.1.243
CuDNN Version  No CUDNN in this machine


In [5]:
def create_symbol(training, n_classes=N_CLASSES):
    # Tensorflow requires a flag for training in dropout
    conv1 = tf.layers.conv2d(X, activation=tf.nn.relu, filters=50, kernel_size=(3, 3), 
                             padding='same', data_format='channels_first')
    conv2 = tf.layers.conv2d(conv1, filters=50, kernel_size=(3, 3), 
                             padding='same', data_format='channels_first')
    pool1 = tf.layers.max_pooling2d(conv2, pool_size=(2, 2), strides=(2, 2), 
                                    padding='valid', data_format='channels_first')
    relu2 = tf.nn.relu(pool1)
    drop1 = tf.layers.dropout(relu2, 0.25, training=training)
    
    conv3 = tf.layers.conv2d(drop1, activation=tf.nn.relu, filters=100, kernel_size=(3, 3), 
                             padding='same', data_format='channels_first')
    conv4 = tf.layers.conv2d(conv3, filters=100, kernel_size=(3, 3), 
                             padding='same', data_format='channels_first')
    pool2 = tf.layers.max_pooling2d(conv4, pool_size=(2, 2), strides=(2, 2), 
                                    padding='valid', data_format='channels_first')
    relu4 = tf.nn.relu(pool2)
    drop2 = tf.layers.dropout(relu4, 0.25, training=training)   
    
    flatten = tf.reshape(drop2, shape=[-1, 100*8*8])
    fc1 = tf.layers.dense(flatten, 512, activation=tf.nn.relu)
    drop3 = tf.layers.dropout(fc1, 0.5, training=training)
    logits = tf.layers.dense(drop3, n_classes, name='output')
    return logits

In [6]:
def init_model(m, y, learning_rate=LR, momentum=MOMENTUM):
    # Single-class labels, don't need dense one-hot
    # Expects unscaled logits, not output of tf.nn.softmax
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=m, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.MomentumOptimizer(learning_rate, momentum)
    training_op = optimizer.minimize(loss)
    return training_op

In [7]:
%%time
# Data into format for library
x_train, x_test, y_train, y_test = cifar_for_library(channel_first=True)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

Preparing train set...
Preparing test set...
(50000, 3, 32, 32) (10000, 3, 32, 32) (50000,) (10000,)
float32 float32 int32 int32
CPU times: user 530 ms, sys: 682 ms, total: 1.21 s
Wall time: 3.77 s


In [8]:
%%time
# Place-holders
X = tf.placeholder(tf.float32, shape=[None, 3, 32, 32])
y = tf.placeholder(tf.int32, shape=[None])
training = tf.placeholder(tf.bool)  # Indicator for dropout layer
# Initialise model
sym = create_symbol(training)

CPU times: user 245 ms, sys: 49.8 ms, total: 295 ms
Wall time: 1.53 s


In [9]:
%%time
model = init_model(sym, y)
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
# Accuracy logging
correct = tf.nn.in_top_k(sym, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

CPU times: user 1.1 s, sys: 1.49 s, total: 2.59 s
Wall time: 25.5 s


In [10]:
%%time
# Main training loop: 57s
EPOCHS=40
for j in range(EPOCHS):
    for data, label in yield_mb(x_train, y_train, BATCHSIZE, shuffle=True):
        sess.run(model, feed_dict={X: data, y: label, training: True})
    # Log
    acc_train = sess.run(accuracy, feed_dict={X: data, y: label, training: True})
    print(j, "Train accuracy:", acc_train)

0 Train accuracy: 0.3203125
1 Train accuracy: 0.46484375
2 Train accuracy: 0.484375
3 Train accuracy: 0.48046875
4 Train accuracy: 0.515625
5 Train accuracy: 0.56640625
6 Train accuracy: 0.6015625
7 Train accuracy: 0.6796875
8 Train accuracy: 0.63671875
9 Train accuracy: 0.72265625
10 Train accuracy: 0.703125
11 Train accuracy: 0.7421875
12 Train accuracy: 0.7421875
13 Train accuracy: 0.75
14 Train accuracy: 0.73046875
15 Train accuracy: 0.7734375
16 Train accuracy: 0.80078125
17 Train accuracy: 0.7421875
18 Train accuracy: 0.77734375
19 Train accuracy: 0.796875
20 Train accuracy: 0.84375
21 Train accuracy: 0.84765625
22 Train accuracy: 0.79296875
23 Train accuracy: 0.7890625
24 Train accuracy: 0.84765625
25 Train accuracy: 0.875
26 Train accuracy: 0.83203125
27 Train accuracy: 0.859375
28 Train accuracy: 0.8671875
29 Train accuracy: 0.8515625
30 Train accuracy: 0.8515625
31 Train accuracy: 0.90625
32 Train accuracy: 0.875
33 Train accuracy: 0.9453125
34 Train accuracy: 0.87890625
35 T

In [11]:
%%time
# Main evaluation loop: 3.22s
n_samples = (y_test.shape[0]//BATCHSIZE)*BATCHSIZE
y_guess = np.zeros(n_samples, dtype=np.int)
y_truth = y_test[:n_samples]
c = 0
for data, label in yield_mb(x_test, y_test, BATCHSIZE):
    pred = tf.argmax(sym,1)
    output = sess.run(pred, feed_dict={X: data, training: False})
    y_guess[c*BATCHSIZE:(c+1)*BATCHSIZE] = output
    c += 1

CPU times: user 2.29 s, sys: 137 ms, total: 2.43 s
Wall time: 2.35 s


In [12]:
print("Accuracy: ", 1.*sum(y_guess == y_truth)/len(y_guess))

Accuracy:  0.7991786858974359
